In [ ]:
#!/usr/bin/python3

# EARTHQUAKE API
# This updates information about earthquakes occuring around the whole world.

import requests
import MySQLdb as mdb
from datetime import datetime

con = mdb.connect(host = 'localhost', 
                  user = 'root', 
                  passwd = 'dwdstudent2015', 
                  charset='utf8', use_unicode=True);

# This gives information for earthquake around the world with a magnitude of 2.5 or higher
url_earthquakes = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_day.geojson'

results = requests.get(url_earthquakes).json() 

# fetch the features of the earthquakes 
data = results["features"]

# Query to create a database
db_name = 'earthquakes'
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8'".format(db=db_name)

# Create a database
cursor = con.cursor()
cursor.execute(create_db_query)
cursor.close()

# Create the a table with the earthquake IDs, magnitude, location, ocurrence time, updated time
# and whether a tsunami has been caused.
cursor = con.cursor()
table_name = 'Earthquakes'

create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                (earthquake_id varchar(250),
                                magnitude float, 
                                place varchar(250), 
                                time datetime,
                                update_time datetime,
                                tsunami int,
                                PRIMARY KEY(earthquake_id)
                                )'''.format(db=db_name, table=table_name)
cursor.execute(create_table_query)
cursor.close()

query_template = '''INSERT IGNORE INTO {db}.{table}(earthquake_id, 
                                            magnitude, 
                                            place, 
                                            time, 
                                            update_time,
                                            tsunami) 
                    VALUES (%s, %s, %s, %s, %s, %s)'''.format(db=db_name, table=table_name)
cursor = con.cursor()

for entry in data:
    earthquake_id = entry['properties']['ids']
    magnitude = entry['properties']['mag']
    place = entry['properties']['place']
    
    #format datetime so that it's readable
    time = datetime.fromtimestamp(
        entry['properties']['time']/1000
    ).strftime('%Y-%m-%d %H:%M:%S')
    
    update_time = datetime.fromtimestamp(
        entry['properties']['updated']/1000
    ).strftime('%Y-%m-%d %H:%M:%S')
    tsunami = entry['properties']['tsunami']

                       
    print("Inserting earthquake", earthquake_id, "from", place)
    query_parameters = (earthquake_id, magnitude, place, time,
                        update_time, tsunami)
    cursor.execute(query_template, query_parameters)

con.commit()
cursor.close()

cur = con.cursor(mdb.cursors.DictCursor)
cur.execute("SELECT * FROM {db}.{table}".format(db=db_name, table=table_name))
rows = cur.fetchall()
cur.close()

for row in rows:
    print("Earthquake ID:", row["earthquake_id"])
    print("Magnitude:", row["magnitude"])
    print("Place:", row["place"])
    print("Time:", row["time"])
    print("Updated Time:", row["update_time"])
    print("Tsunami Status:", row["tsunami"])
    print("=============================================")







